# gpuramdrive

In [13]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [10]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [14]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# upload


In [90]:
from google.colab import files
uploaded=files.upload()

Saving test_3시간평균_weight.csv to test_3시간평균_weight.csv


In [15]:
import io

In [89]:
train = pd.read_csv(io.StringIO(uploaded['train_3시간평균_weight.csv'].decode('utf-8')))

In [91]:
test = pd.read_csv(io.StringIO(uploaded['test_3시간평균_weight.csv'].decode('utf-8')))

# model

In [ ]:
input_df_list = []
for i in range(0, 28):
    input_df = pd.read_csv(all_input_list[i])
    input_df = input_df[[col for col in input_df.columns if not '누적' in col]]
    input_df['obs_time'] = [ num for num in range(0, 24) ] * 28
    input_df.drop(columns=['시간당총광량'], inplace=True)
    input_df['case'] = i + 1
    input_df_list.append(input_df)

# input_df_list
input_df_total = pd.concat(input_df_list, axis=0)

input_df_total.loc[input_df_total['시간당분무량'] < 0, '시간당분무량' ] = (-1) * input_df_total.loc[input_df_total['시간당분무량'] < 0, '시간당분무량' ]

input_df_total.loc[input_df_total['시간당백색광량'] < 0,  '시간당백색광량'] = np.nan
input_df_total.loc[input_df_total['시간당적색광량'] < 0,  '시간당적색광량'] = np.nan
input_df_total.loc[input_df_total['시간당청색광량'] < 0,  '시간당청색광량'] = np.nan

pd.set_option('display.max_rows', None)
input_df_total.loc[(input_df_total["DAT"]==4)&(input_df_total["case"]==5),:]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
input_df_total.iloc[:, :-1] = scaler.fit_transform(input_df_total.iloc[:, :-1].copy())
input_df_total

new_input_list = []
for case in range(1, 29):
    new_input_list.append(input_df_total.loc[input_df_total['case']==case, :'시간당청색광량'])
    
new_input_list[0]

new_target_list = []
for i in range(0, 28):
    target_df = pd.read_csv(all_target_list[i])
    target_df['diff'] = target_df['predicted_weight_g'].diff()
    target_df.loc[0, 'diff'] = target_df.loc[0, 'predicted_weight_g']
    # target_df.drop(columns=['predicted_weight_g'], inplace=True)
    # target_df.rename(columns={'diff':'predicted_weight_g'}, inplace=True)
    new_target_list.append(target_df)

    
data_list = []
label_list = []

for input_df, target_df in tqdm(zip(new_input_list, new_target_list)):
    
    input_length = int(len(input_df)/24)
    target_length = int(len(target_df))
    
    for idx in range(target_length):
        time_series = input_df[24*idx:24*(idx+1)].values
        data_list.append(time_series)
    for label in target_df["diff"]:
        label_list.append(label)

x_data = np.array(data_list)
y_data = np.array(label_list)

x_data.shape, y_data.shape


from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.2, shuffle=True)

input_shape = (x_train[0].shape[0], x_train[0].shape[1])

model = Sequential()
model.add(Conv1D(32, 6, input_shape=input_shape))
model.add(Bidirectional(LSTM(32, return_sequences=True)))
model.add(Bidirectional(LSTM(16)))
model.add(Dense(1, activation = 'relu'))

optimizer = tf.optimizers.Adam(0.001)

model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

# es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.fit(x_train, y_train, 
          validation_data=(x_valid, y_valid),
          epochs=100, 
          batch_size=8,
        #   callbacks=[es],
         )

test_input_list = sorted(glob.glob(os.path.join(DATA_DIR, 'test_input', '*.csv')))
test_target_list = sorted(glob.glob(os.path.join(DATA_DIR, 'test_target', '*.csv')))
test_target_list

for test_input_path, test_target_path in zip(test_input_list, test_target_list):
    input_df = pd.read_csv(test_input_path)
    input_df = input_df[[col for col in input_df.columns if not '누적' in col]]
    input_df['obs_time'] = [ num for num in range(0, 24) ] * 28
    input_df.drop(columns=['시간당총광량'], inplace=True)

    input_df.loc[input_df['시간당분무량'] < 0, '시간당분무량' ] = (-1) * input_df.loc[input_df['시간당분무량'] < 0, '시간당분무량' ]
    input_df.loc[input_df['시간당백색광량'] < 0,  '시간당백색광량'] = np.nan
    input_df.loc[input_df['시간당적색광량'] < 0,  '시간당적색광량'] = np.nan
    input_df.loc[input_df['시간당청색광량'] < 0,  '시간당청색광량'] = np.nan
    input_df['시간당백색광량'] = input_df['시간당백색광량'].interpolate()
    input_df['시간당적색광량'] = input_df['시간당적색광량'].interpolate()
    input_df['시간당청색광량'] = input_df['시간당청색광량'].interpolate()

    input_df.iloc[:, :] = scaler.transform(input_df.iloc[:, :].copy())   

    input_length = int(len(input_df)/24)
    
    data_list = []
    for idx in range(28):
        time_series = input_df[24*idx:24*(idx+1)].values
        data_list.append(time_series)
        
    pred = model.predict(np.array(data_list)).tolist()  

    submit_df = pd.read_csv(test_target_path)
    print(submit_df.shape)
    submit_df['predicted_weight_g'] = np.ravel(pred)
    submit_df['predicted_weight_g'] = submit_df['predicted_weight_g'].cumsum()
    submit_df.to_csv(test_target_path, index=False)

    import zipfile
os.chdir("./data/test_target/")
# print(os.getcwd())
submission = zipfile.ZipFile("../submission.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    print(path)
    submission.write(path)
submission.close()

In [58]:
import pandas as pd
train = pd.read_csv('drive/MyDrive/dacon/data/train_input/CASE_02.csv')
train

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
0,0,0,-0.058436,-0.290376,1.312568,1.360304,-0.849115,-0.913229,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
1,0,1,-0.188055,-0.152841,1.321495,1.358284,-0.849115,-0.904295,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
2,0,2,-0.636864,-0.036465,1.194178,1.326899,-0.849115,-0.895361,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
3,0,3,-0.863590,0.069475,1.221836,1.387595,-0.849115,-0.886427,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
4,0,4,-0.779148,-0.140832,1.168714,1.402227,-0.849115,-0.877493,-0.775478,-0.847469,-0.535571,-0.529592,-0.502909,-0.510168,-1.002462,-1.080641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,27,19,-0.037326,0.884752,0.996617,-1.446825,-0.849115,-0.636281,-0.775478,-0.847469,-0.111823,1.168255,-0.026632,1.481377,-0.736906,0.084665
668,27,20,-0.025082,1.065106,1.348129,-1.446825,-0.849115,-0.627348,-0.775478,-0.847469,-0.535571,1.168255,-0.502909,1.481377,-1.002462,0.084665
669,27,21,-0.011571,1.100705,1.616957,-1.446825,-0.849115,-0.618414,-0.775478,-0.847469,-0.535571,1.168255,-0.502909,1.481377,-1.002462,0.084665
670,27,22,-0.068147,1.138449,1.557250,-1.446825,-0.849115,-0.609480,-0.775478,-0.847469,-0.535571,1.168255,-0.502909,1.481377,-1.002462,0.084665


In [23]:
import catboost
from sklearn.metrics import mean_squared_error

In [42]:
train=train.fillna(0)

In [43]:
test=test.fillna(0)

In [94]:
corr_df = train.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
s = corr_df.unstack()
성장 = pd.DataFrame(s['predicted_weight_g']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
성장.sort_values(by=[0],ascending=False).reset_index().to_csv('target값이랑corr.csv',index=False,encoding='utf-8-sig')
under_1per = 성장[성장[0]<0.1].index
under_1per

Index(['ec관측치21+23', 'ec관측치18+20', 'ec관측치15+17', 'ec관측치12+14', 'ec관측치9+11',
       '시간당청색광량6+8', 'ec관측치6+8', '시간당청색광량9+11', '시간당청색광량12+14', '시간당청색광량15+17',
       'ec관측치3+5', '시간당청색광량18+20', 'ec관측치0+2', '시간당백색광량0+2', '시간당백색광량3+5',
       '시간당백색광량21+23', '시간당적색광량0+2', '시간당적색광량3+5', '시간당적색광량21+23',
       '시간당청색광량0+2', '시간당청색광량3+5', '시간당청색광량21+23'],
      dtype='object')

In [95]:
성장

,0
일간누적청색광량21,0.21
일간누적적색광량11,0.21
일간누적청색광량22,0.21
일간누적청색광량23,0.21
일간누적적색광량12,0.21
...,...
시간당적색광량3+5,0.00
시간당적색광량21+23,0.00
시간당청색광량0+2,0.00
시간당청색광량3+5,0.00


In [65]:
drop_column = train.copy()
drop_column = drop_column.drop(under_1per,axis=1)
# drop_columntt = test.copy()
# drop_columntt = drop_columntt.drop(under_1per,axis=1)

In [ ]:
drop_columntt = test.copy()
drop_columntt = drop_columntt.drop(under_1per,axis=1)
drop_columntt.drop(['predicted_weight_g'],axis=1)

In [92]:
X = train.drop(['predicted_weight_g'],axis=1)
y = train['predicted_weight_g']
model = catboost.CatBoostRegressor(verbose=0,random_state=404)
model.fit(X,y)

In [93]:
pred = model.predict(test.drop(['predicted_weight_g'],axis=1))
pred

array([26.58539761, 28.46123904, 31.31561895, 33.82420957, 35.32938326,
       37.60402244, 39.19471044, 40.57154035, 42.83728767, 41.49363367,
       44.99257115, 44.87474725, 47.95953572, 48.9625183 , 50.02911792,
       51.76434864, 54.09107541, 37.77782603, 40.92114282, 43.11437632,
       39.96694317, 44.15825502, 43.42044569, 42.33656508, 49.12863669,
       57.98113863, 56.58989152, 60.83820726, 26.58539761, 28.46123904,
       31.31561895, 33.82420957, 35.32938326, 37.60402244, 39.19471044,
       40.57154035, 42.83728767, 41.49363367, 44.99257115, 44.87474725,
       47.95953572, 48.9625183 , 50.02911792, 51.76434864, 54.09107541,
       37.77782603, 40.92114282, 43.11437632, 39.96694317, 44.15825502,
       43.42044569, 42.33656508, 49.12863669, 57.98113863, 56.58989152,
       60.83820726, 26.58539761, 28.46123904, 31.31561895, 33.82420957,
       35.32938326, 37.60402244, 39.19471044, 40.57154035, 42.83728767,
       41.49363367, 44.99257115, 44.87474725, 47.95953572, 48.96

In [115]:
drop_column

,내부온도관측치0+3,내부온도관측치1+4,내부온도관측치2+5,내부온도관측치3+6,내부온도관측치4+7,내부온도관측치5+8,내부온도관측치6+9,내부온도관측치7+10,내부온도관측치8+11,내부온도관측치9+12,...,일간누적총광량20,일간누적총광량21,일간누적총광량22,일간누적총광량23,day_white_weight_max,day_light_weight_max,day_white_weight_mean,day_light_weight_mean,DAT,predicted_weight_g
0,-0.633487,-0.610265,-0.687620,-0.392918,0.013670,0.453613,0.958153,1.256233,1.453827,1.640866,...,1.684933,1.684933,1.684933,1.684933,2.122115,1.684933,1.434604,1.132691,1,0.167719
1,-0.319614,-0.414611,-0.560113,-0.174636,0.405058,1.062017,1.768796,2.083342,2.288114,2.459531,...,1.691674,1.691674,1.691674,1.691674,4.259252,3.383348,2.878633,2.273928,2,0.181787
2,-0.006117,-0.117158,-0.297499,0.085024,0.660918,1.311543,1.874348,1.783995,1.418784,0.978419,...,1.685171,1.685171,1.685171,1.685171,6.367963,5.055514,4.300346,3.394364,3,0.265921
3,-0.596495,-0.637871,-0.774173,-0.376029,0.196065,0.542277,0.829715,0.708541,0.445082,0.425661,...,1.691674,1.691674,1.691674,1.691674,8.518503,6.766695,5.757266,4.547857,4,0.423650
4,-0.618958,-0.621069,-0.634422,-0.383629,-0.037417,0.303306,0.690472,0.872867,1.039639,1.190790,...,1.686319,1.686319,1.686319,1.686319,10.619380,8.431597,7.173352,5.663013,5,0.475272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,1.738819,1.557691,1.367274,1.214434,1.117326,1.178125,1.391763,1.741774,2.286847,2.683302,...,0.086482,0.086482,0.086482,0.086482,0.000000,2.075566,0.000000,1.977012,24,64.875499
780,-1.118274,-1.198072,-1.292225,-1.374556,-1.497841,-1.480108,-1.218338,-0.950826,-0.501595,-0.148627,...,0.086630,0.086630,0.086630,0.086630,0.000000,2.165758,0.000000,2.061719,25,74.002614
781,-2.746737,-2.957420,-3.057906,-2.903377,-2.205887,-0.932924,0.301195,1.256233,1.541647,1.357141,...,0.086085,0.086085,0.086085,0.086085,0.000000,2.238212,0.000000,2.130441,26,76.342275
782,-2.272595,-2.957420,-3.057906,-2.903377,-2.205887,-0.932924,0.301195,1.256233,1.541647,1.357141,...,0.086085,0.086085,0.086085,0.086085,0.000000,2.324297,0.000000,2.212381,27,82.621245


In [71]:
train

,내부온도관측치0+2,내부온도관측치1+3,내부온도관측치2+4,내부온도관측치3+5,내부온도관측치4+6,내부온도관측치5+7,내부온도관측치6+8,내부온도관측치7+9,내부온도관측치8+10,내부온도관측치9+11,...,일간누적총광량16,일간누적총광량17,일간누적총광량18,일간누적총광량19,일간누적총광량20,일간누적총광량21,일간누적총광량22,일간누적총광량23,DAT,predicted_weight_g
0,25.417897,25.436230,25.340000,25.373889,25.734444,26.257222,26.841667,27.151667,27.369445,27.573333,...,210911.3740,230142.8960,249374.4180,268605.9400,269376.2938,269376.2938,269376.2938,269376.2938,1,0.167719
1,25.790345,25.680345,25.497222,25.438333,25.981111,26.758889,27.631667,27.995556,28.189444,28.372222,...,211220.7840,230452.3060,249683.8280,268915.3500,270032.8895,270032.8895,270032.8895,270032.8895,2,0.181787
2,26.096742,25.995631,25.777778,25.685556,26.216667,27.019445,27.895000,28.040000,27.694444,27.160556,...,210727.0981,229958.6201,249190.1421,268421.6641,269399.5381,269399.5381,269399.5381,269399.5381,3,0.265921
3,25.514350,25.438795,25.269444,25.243889,25.804445,26.536111,27.001667,26.830442,26.478776,26.408220,...,211220.7840,230452.3060,249683.8280,268915.3500,270032.8895,270032.8895,270032.8895,270032.8895,4,0.423650
4,25.444793,25.398682,25.398333,25.401111,25.754444,26.166667,26.632222,26.814444,26.952222,27.135555,...,210838.9167,230070.4387,249301.9607,268533.4827,269511.3567,269511.3567,269511.3567,269511.3567,5,0.475272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,27.838889,27.663333,27.468889,27.275000,27.148889,27.083889,27.220000,27.508333,27.970555,28.542778,...,94625.6875,104128.5225,111283.0298,113682.8983,113682.8983,113682.8983,113682.8983,113682.8983,24,64.875499
780,24.953889,24.858333,24.813333,24.735555,24.616667,24.463889,24.566111,24.941111,25.336444,25.751444,...,94590.2532,104093.0882,111265.2685,113697.3500,113697.3500,113697.3500,113697.3500,113697.3500,25,74.002614
781,23.405000,23.179445,23.028889,22.966667,23.198889,24.168333,25.825000,27.183333,27.754445,27.424444,...,94519.3846,104022.2196,111221.8251,113644.2427,113644.2427,113644.2427,113644.2427,113644.2427,26,76.342275
782,24.028889,23.179445,23.028889,22.966667,23.198889,24.168333,25.825000,27.183333,27.754445,27.424444,...,94519.3846,104022.2196,111221.8251,113644.2427,113644.2427,113644.2427,113644.2427,113644.2427,27,82.621245


In [73]:
import pandas as pd
train = pd.read_csv('drive/MyDrive/dacon/data/train_3시간단위.csv')
train

,내부온도관측치0+1,내부온도관측치0+1+2,내부온도관측치1+2,내부온도관측치1+2+3,내부온도관측치2+3,내부온도관측치2+3+4,내부온도관측치3+4,내부온도관측치3+4+5,내부온도관측치4+5,내부온도관측치4+5+6,...,일간누적총광량18+19,일간누적총광량18+19+20,일간누적총광량19+20,일간누적총광량19+20+21,일간누적총광량20+21,일간누적총광량20+21+22,일간누적총광량21+22,일간누적총광량21+22+23,DAT,predicted_weight_g
0,50.980357,76.253691,50.953691,76.308691,50.628333,76.020000,50.746667,76.121667,50.766667,77.203333,...,517980.3580,787356.6518,537982.2338,807358.5276,538752.5876,808128.8814,538752.5876,808128.8814,1,0.167719
1,51.787700,77.371034,51.564368,77.041034,51.060000,76.491667,50.908333,76.314999,50.838333,77.943333,...,518599.1780,788632.0675,538948.2395,808981.1290,540065.7790,810098.6685,540065.7790,810098.6685,2,0.181787
2,52.411893,78.290226,52.191892,77.986892,51.673332,77.333333,51.455000,77.056667,51.261668,78.650001,...,517611.8062,787011.3443,537821.2022,807220.7403,538799.0762,808198.6143,538799.0762,808198.6143,3,0.265921
3,51.221384,76.543050,51.104717,76.316384,50.533333,75.808333,50.486667,75.731667,50.520000,77.413334,...,518599.1780,788632.0675,538948.2395,808981.1290,540065.7790,810098.6685,540065.7790,810098.6685,4,0.423650
4,50.932712,76.334378,50.864378,76.196045,50.733333,76.195000,50.793333,76.203333,50.871667,77.263333,...,517835.4434,787346.8001,538044.8394,807556.1961,539022.7134,808534.0701,539022.7134,808534.0701,5,0.475272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,60.731044,91.499377,61.131045,91.776045,61.413334,92.360000,61.591667,92.418333,61.773332,92.908333,...,206867.0919,310305.2790,206876.3742,310314.5613,206876.3742,310314.5613,206876.3742,310314.5613,24,64.875499
780,47.397241,71.215575,47.535575,71.195574,47.478333,71.136666,47.318333,70.880000,47.220000,70.950000,...,358932.7866,538402.2740,358938.9748,538408.4622,358938.9748,538408.4622,358938.9748,538408.4622,25,74.002614
781,48.104195,72.312529,48.284195,72.385862,48.310000,72.323333,48.115000,72.015000,47.913333,71.851666,...,358855.7477,538288.2627,358865.0300,538297.5450,358865.0300,538297.5450,358865.0300,538297.5450,26,76.342275
782,48.035000,72.093334,48.043334,72.023334,48.038334,72.005000,47.946667,71.866667,47.886667,71.836666,...,358733.8254,538106.9263,358746.2018,538119.3027,358746.2018,538119.3027,358746.2018,538119.3027,27,82.621245


In [77]:
import catboost
X = train.drop(['predicted_weight_g'],axis=1)
y = train['predicted_weight_g']
model= {}
for f in range(0,27):
      train_idx = [i for i in range(0,784)]
      valid_idx = train_idx[f*28:f*28+28]
      for idx in valid_idx:
        train_idx.remove(idx)
      print(f'===================================={f+1}============================================')
      #print(min(valid_idx),m
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
      
      xgb_model = catboost.CatBoostRegressor( verbose=0,random_state=404)
      xgb_model.fit(x_train, y_train)
      
      y_pred = xgb_model.predict(x_val)
      print(y_pred)
      rmse = mean_squared_error(y_val, y_pred)**0.5
      print(f"{f + 1} Fold rmse = {rmse}")
      model[f] = xgb_model
      print(f'================================================================================\n\n')

====================================1============================================
[  2.6758695    3.47768624   3.96263929   4.627987     5.63949723
   7.05791484   8.63626877   5.27950517  11.97385439  14.54523313
   8.64199016  21.57050123  13.05927372  31.25929658  35.83002572
  41.33794895  46.92751373  52.38323014  58.71234847  65.63281244
  72.39255281  79.1687044   85.35736889  91.02489826  96.54792862
 101.04731246 106.68571274 111.83198984]
1 Fold rmse = 17.34585928528509


====================================2============================================
[  2.61985065   3.41031527   3.89697066   4.55809742   5.56130816
   7.06210447   8.63001753   5.33687158  12.03904107  14.61528246
   8.81215216  21.69665661  13.40214523  31.42609005  36.09766748
  41.7058697   47.39114112  52.94899556  59.42656323  66.4644947
  73.49611632  80.68322364  86.95228665  92.75513244  98.4687548
 102.88233765 108.39551171 112.94244727]
2 Fold rmse = 30.01402655631224


============================

KeyboardInterrupt: ignored

In [47]:
test

,내부온도관측치2+5,내부온도관측치3+6,내부온도관측치4+7,내부온도관측치5+8,내부온도관측치6+9,내부온도관측치7+10,내부온도관측치8+11,내부온도관측치9+12,내부온도관측치10+13,내부온도관측치11+14,...,일간누적청색광량20,일간누적청색광량21,일간누적청색광량22,일간누적청색광량23,day_water_weight_max,day_white_weight_max,day_red_weight_max,day_blue_weight_max,DAT,predicted_weight_g
0,-0.141013,-0.035704,0.133732,0.361969,0.515220,0.637534,0.766200,0.803488,0.901626,1.006730,...,-0.638521,-0.638521,-0.638521,-0.638521,8.939036,1.591620,0.000000,0.000000,1,0
1,0.426917,0.504771,0.679945,1.014310,1.381867,1.677305,1.804126,1.764584,1.677305,1.646778,...,-0.638521,-0.638521,-0.638521,-0.638521,12.806160,3.184102,0.000000,0.000000,2,0
2,0.714570,0.683018,0.737926,0.842006,0.986242,1.126380,1.201162,1.242753,1.276558,1.315895,...,-0.638521,-0.638521,-0.638521,-0.638521,25.977855,4.779635,0.000000,0.000000,3,0
3,0.263422,0.280427,0.388809,0.634051,0.894455,1.173503,1.377974,1.481439,1.535732,1.567489,...,-0.638521,-0.638521,-0.638521,-0.638521,35.928298,6.371255,0.000000,0.000000,4,0
4,0.382867,0.423229,0.560499,0.828483,0.968827,0.971081,0.824386,0.537348,0.384097,0.327959,...,-0.638521,-0.638521,-0.638521,-0.638521,30.589745,7.966224,0.000000,0.000000,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,1.056106,1.113063,1.149737,1.226977,1.087043,0.818444,0.560294,0.182904,-0.009330,-0.120376,...,-0.041720,-0.041720,-0.041720,-0.041720,535.009358,0.000000,2.601229,0.000000,24,0
136,-2.086358,-1.971625,-1.686020,-1.348786,-0.997006,-0.614493,-0.349787,-0.128516,0.083741,0.151966,...,0.390325,0.390325,0.390325,0.390325,37.622787,11.663376,20.088400,9.758121,25,0
137,-2.035752,-1.936590,-1.669630,-1.329937,-0.975698,-0.771842,-0.707919,-0.720417,-0.726358,-0.706280,...,0.389942,0.389942,0.389942,0.389942,30.401906,12.110077,20.821857,10.138484,26,0
138,-2.025303,-1.911390,-1.665328,-1.278307,-0.935132,-0.709763,-0.637849,-0.678416,-0.684153,-0.697675,...,0.396302,0.396302,0.396302,0.396302,32.733252,12.665400,21.772656,10.700155,27,0


In [46]:
testX = test.drop(['predicted_weight_g'],axis=1)
pc = [0]*140
pc = model[0].predict(testX)
pc # 4444

array([26.9754648 , 28.37486009, 28.13342746, 28.24143575, 24.94480723,
       28.04876109, 27.83002886, 28.34531494, 28.50938086, 28.17496584,
       27.58571718, 27.58182681, 27.75571371, 25.66109153, 31.13102138,
       27.23717704, 26.42588024, 24.47002882, 25.59609361, 26.2780595 ,
       25.46736732, 26.04564919, 25.95334032, 25.18133834, 28.15405043,
       26.7846155 , 27.61481777, 26.47461629, 26.9754648 , 28.37486009,
       28.13342746, 28.24143575, 24.94480723, 28.04876109, 27.83002886,
       28.34531494, 28.50938086, 28.17496584, 27.58571718, 27.58182681,
       27.75571371, 25.66109153, 31.13102138, 27.23717704, 26.42588024,
       24.47002882, 25.59609361, 26.2780595 , 25.46736732, 26.04564919,
       25.95334032, 25.18133834, 28.15405043, 26.7846155 , 27.61481777,
       26.47461629, 26.9754648 , 28.37486009, 28.13342746, 28.24143575,
       24.94480723, 28.04876109, 27.83002886, 28.34531494, 28.50938086,
       28.17496584, 27.58571718, 27.58182681, 27.75571371, 25.66

In [38]:
testX = test.drop(['predicted_weight_g'],axis=1)
pc=[0]*140
for fold in range(27):
    pc += model[1].predict(testX)/27
pc # 4444

KeyError: ignored

In [36]:
test

,내부온도관측치2+5,내부온도관측치3+6,내부온도관측치4+7,내부온도관측치5+8,내부온도관측치6+9,내부온도관측치7+10,내부온도관측치8+11,내부온도관측치9+12,내부온도관측치10+13,내부온도관측치11+14,...,일간누적청색광량20,일간누적청색광량21,일간누적청색광량22,일간누적청색광량23,day_water_weight_max,day_white_weight_max,day_red_weight_max,day_blue_weight_max,DAT,predicted_weight_g
0,-0.141013,-0.035704,0.133732,0.361969,0.515220,0.637534,0.766200,0.803488,0.901626,1.006730,...,-0.638521,-0.638521,-0.638521,-0.638521,8.939036,1.591620,NaN,NaN,1,0
1,0.426917,0.504771,0.679945,1.014310,1.381867,1.677305,1.804126,1.764584,1.677305,1.646778,...,-0.638521,-0.638521,-0.638521,-0.638521,12.806160,3.184102,NaN,NaN,2,0
2,0.714570,0.683018,0.737926,0.842006,0.986242,1.126380,1.201162,1.242753,1.276558,1.315895,...,-0.638521,-0.638521,-0.638521,-0.638521,25.977855,4.779635,NaN,NaN,3,0
3,0.263422,0.280427,0.388809,0.634051,0.894455,1.173503,1.377974,1.481439,1.535732,1.567489,...,-0.638521,-0.638521,-0.638521,-0.638521,35.928298,6.371255,NaN,NaN,4,0
4,0.382867,0.423229,0.560499,0.828483,0.968827,0.971081,0.824386,0.537348,0.384097,0.327959,...,-0.638521,-0.638521,-0.638521,-0.638521,30.589745,7.966224,NaN,NaN,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,1.056106,1.113063,1.149737,1.226977,1.087043,0.818444,0.560294,0.182904,-0.009330,-0.120376,...,-0.041720,-0.041720,-0.041720,-0.041720,535.009358,NaN,2.601229,NaN,24,0
136,-2.086358,-1.971625,-1.686020,-1.348786,-0.997006,-0.614493,-0.349787,-0.128516,0.083741,0.151966,...,0.390325,0.390325,0.390325,0.390325,37.622787,11.663376,20.088400,9.758121,25,0
137,-2.035752,-1.936590,-1.669630,-1.329937,-0.975698,-0.771842,-0.707919,-0.720417,-0.726358,-0.706280,...,0.389942,0.389942,0.389942,0.389942,30.401906,12.110077,20.821857,10.138484,26,0
138,-2.025303,-1.911390,-1.665328,-1.278307,-0.935132,-0.709763,-0.637849,-0.678416,-0.684153,-0.697675,...,0.396302,0.396302,0.396302,0.396302,32.733252,12.665400,21.772656,10.700155,27,0
